In [8]:
#required libraries
import numpy as np
import pygame
import tensorflow as tf

In [9]:
# global constants
SCREEN_WIDTH, SCREEN_HEIGHT = 1280, 720
BG_COLOR = (120, 120, 120)

DRAW_COLOR = (50, 50, 50)
DRAW_RADIUS = 2
ERASE_RADIUS = 5

CAR_WIDTH, CAR_HEIGHT = 40, 20
DEFAULT_START_X, DEFAULT_START_Y = 897, 380
DEFAULT_START_ANGLE = 78
DEFAULT_START_SPEED = 0

ACCELERATION = 0.1
FRICTION = 0.05
TURN_ANGLE = 2

TRACK_SAVE_PATH = "track.png"
CAR_IMAGE_PATH = "Track_images/car.png"
TRACK_IMAGE_PATH = r"Track_images\silverstone_track.png"


FOR making th track drawing 
left ckick for drawing the track
right click for eraing the track

In [10]:
def draw_track():
    """
    Function to draw the racing track.
    Left mouse button to draw, right mouse button to erase.
    Press 'S' to save the track as an image.
    """
    pygame.init()
    pygame.display.set_caption("Drawing the track, press 'S' to save as image.")
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    screen.fill(BG_COLOR)

    running, drawing, erase = True, False, False

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if event.button == 1:
                    drawing = True
                elif event.button == 3:
                    erase = True
            elif event.type == pygame.MOUSEBUTTONUP:
                if event.button == 1:
                    drawing = False
                elif event.button == 3:
                    erase = False
            elif event.type == pygame.MOUSEMOTION:
                mouse_pos = pygame.mouse.get_pos()
                if drawing:
                    pygame.draw.circle(screen, DRAW_COLOR, mouse_pos, DRAW_RADIUS)
                elif erase:
                    pygame.draw.circle(screen, BG_COLOR, mouse_pos, ERASE_RADIUS)
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_s:
                pygame.image.save(screen, TRACK_SAVE_PATH)

        pygame.display.update()
    pygame.quit()


In [11]:
class Car:
    """
    Class representing a car in the racing simulation.
    """
    def __init__(self, image_path, x, y, angle=0, speed=0):
        image = pygame.image.load(image_path).convert_alpha()
        self.image = pygame.transform.scale(image, (CAR_WIDTH, CAR_HEIGHT))
        self.x, self.y = x, y
        self.angle = angle
        self.speed = speed

    def move(self):
        rad = np.radians(self.angle)
        self.x += self.speed * np.cos(rad)
        self.y -= self.speed * np.sin(rad)

    def draw(self, screen):
        rotated_img = pygame.transform.rotate(self.image, -self.angle)
        rect = rotated_img.get_rect(center=(self.x, self.y))
        screen.blit(rotated_img, rect.topleft)


In [12]:
def ray_casting(car,screen,track_surface):
    # Ray casting implementation
    sensor_distance=[]
    sensor_endpoint=[]
    sensor_angle=[-45,0,45]
    for angle in sensor_angle:
        ray_angle=car.angle+angle
        ray_x,ray_y=car.x,car.y
        distance=0
        max_distance=200
        while distance < max_distance:
            rad=np.radians(ray_angle)+np.radians(car.angle+45)
            ray_x+=np.cos(rad)
            ray_y-=np.sin(rad)
            distance+=1
            if not (0<=ray_x<SCREEN_WIDTH or 0<=ray_y<SCREEN_HEIGHT):
                break
            pixel_color=track_surface.get_at((int(ray_x),int(ray_y)))
            if pixel_color==DRAW_COLOR:
                break
        sensor_distance.append(distance)
        sensor_endpoint.append((ray_x,ray_y))
    return sensor_distance,sensor_endpoint


In [ ]:
def run_simulation(start_x=DEFAULT_START_X, 
                   start_y=DEFAULT_START_Y, 
                   start_angle=DEFAULT_START_ANGLE, 
                   start_speed=DEFAULT_START_SPEED):
    pygame.init()
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    clock = pygame.time.Clock()
    pygame.display.set_caption("Car Simulation")

    track_image = pygame.image.load(TRACK_IMAGE_PATH).convert()
    car = Car(CAR_IMAGE_PATH, start_x, start_y, start_angle, start_speed)

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        screen.fill(BG_COLOR)
        screen.blit(track_image, (0, 0))

        # manage car speed and direction
        keys=pygame.key.get_pressed()
        car.speed-=FRICTION
        if abs(car.speed)<FRICTION:
            car.speed=0
        if keys[pygame.K_UP]:
            car.speed+=ACCELERATION
        if keys[pygame.K_DOWN]:
            car.speed-=ACCELERATION
        if car.speed != 0:
            if keys[pygame.K_LEFT]:
                car.angle+=TURN_ANGLE
            if keys[pygame.K_RIGHT]:
                car.angle-=TURN_ANGLE
                
        # Ray casting
        distance,sensor_endpoint=ray_casting(car,screen,track_image)
        font=pygame.font.Font(None,22)
        text_surface=font.render(f"Distances: {distance}",True,(255,255,255))
        screen.blit(text_surface,(50,50))
        for endpoint in sensor_endpoint:
            pygame.draw.line(screen,(0,255,0),(car.x,car.y),endpoint,1)
            
        car.move()
        car.draw(screen)

        
        # collision detection
        ROAD_COLOR = DRAW_COLOR

        try:
            car_center_pos = (int(car.x), int(car.y))
            pixel_color = screen.get_at(car_center_pos)
            if pixel_color == ROAD_COLOR:
                font = pygame.font.Font(None, 22)  
                text_surface = font.render("Car has crashed!", True, (255, 255, 255))  
                screen.blit(text_surface, (50, 65))  
                # Reset car to start position and angle
                car.x, car.y = start_x, start_y
                car.angle = start_angle
                car.speed = start_speed

        except IndexError:
            # Car went off screen
            font = pygame.font.Font(None, 22)  
            text_surface = font.render("Car has crashed!", True, (255, 255, 255)) 
            screen.blit(text_surface, (50, 65))  
            # Reset car to start position and angle
            # car.x, car.y = start_x, start_y
            # car.angle = start_angle
            # car.speed = start_speed

        pygame.display.update()
        clock.tick(60)

    pygame.quit()


In [ ]:
if __name__ == "__main__":
    # draw_track()
    run_simulation()


IndexError: pixel index out of range

: 